<a href="https://colab.research.google.com/github/wolffg7/Sentiment_Prediction/blob/main/SentimentProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Stage 1: Importing relevant libraries and loading the sst5 dataset
  # See https://huggingface.co/datasets/SetFit/sst5 for documentation on the dataset

import numpy as np 
import pandas as pd 

from tensorflow.keras.preprocessing.text import Tokenizer

from datasets import load_dataset
dataset = load_dataset("SetFit/sst5") 

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Viewing the structure of the dataset and practicing indexing into it

print(dataset) 

print(dataset['train'][0]['text']) 

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
})
a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films


In [ ]:
# Stage 2: Preparing the train, test, and validation datasets for tokenization
  # Separate individual sentences from movie review snippets

all_sentences = [] 
#train set fetch 
for i in range(8543): 
        cur_sentence = dataset['train'][i]['text'] 
        all_sentences.append(cur_sentence) 

#test set fetch 
for i in range(2209): 
        cur_sentence = dataset['test'][i]['text'] 
        all_sentences.append(cur_sentence) 

#validation set fetch 
for i in range(1100): 
        cur_sentence = dataset['validation'][i]['text'] 
        all_sentences.append(cur_sentence) 
print(len(all_sentences))
all_sentences[:5]

11852


['a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films',
 'apparently reassembled from the cutting-room floor of any given daytime soap .',
 "they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science-fiction elements of bug-eyed monsters and futuristic women in skimpy clothes .",
 'the entire movie is filled with deja vu moments .',
 'this is a visually stunning rumination on love , memory , history and the war between art and commerce .']

In [ ]:
# Stage 3: Removing stop words from the dataset

import nltk
from nltk.corpus import stopwords

# print(stopwords.words('english'))
# english_stop_words = stopwords.words('english') 

english_stop_words_changed = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 
 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 
 'most', 'other', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'now',]



def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words_changed])
        )
    return removed_stop_words

sentences_nosw = remove_stop_words(all_sentences) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
 # Stage 4: Initialization tokenizer

#tokenize words in all sentences 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_nosw)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words) 

Total words:  17788


In [ ]:
# Stage 5: Perform tokenization

input_sequences = []
for line in all_sentences:
    # Transform sentences into tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # Build sequences for each sentence
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:10]

['a stirring', 'a stirring funny', 'a stirring funny and', 'a stirring funny and finally', 'a stirring funny and finally transporting']


[[443, 1485],
 [443, 1485, 27],
 [443, 1485, 27, 261],
 [443, 1485, 27, 261, 492],
 [443, 1485, 27, 261, 492, 4520],
 [443, 1485, 27, 261, 492, 4520, 1486],
 [443, 1485, 27, 261, 492, 4520, 1486, 7057],
 [443, 1485, 27, 261, 492, 4520, 1486, 7057, 170],
 [443, 1485, 27, 261, 492, 4520, 1486, 7057, 170, 623],
 [443, 1485, 27, 261, 492, 4520, 1486, 7057, 170, 623, 261]]

In [ ]:
# Stage 6: Apply padding to sequences 

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  443, 1485], dtype=int32)